In [ ]:
%cd ..

In [ ]:
import glob
import os
from datetime import datetime
import json

import polars as pl
from plotly.graph_objects import Figure

logs = glob.glob("runs/amip/*/output.log")

dates = []
sypds = []
run_locs = []

for log in logs:
    date = datetime.fromisoformat(log.split("/")[-2])
    dates.append(date)
    run_locs.append("calkit")
    with open(log, "r") as f:
        lines = f.readlines()
    sypd = None
    found_step_40 = False
    for line in lines:
        if (
            "n_steps_completed = 40" in line
            or "n_steps_completed = 41" in line
        ):
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

# Now read any fetched buildkite logs
bk_jobs = glob.glob("buildkite/*.json")
for job_fpath in bk_jobs:
    with open(job_fpath) as f:
        job = json.load(f)
    date = datetime.fromisoformat(job["created_at"])
    dates.append(date)
    run_locs.append("buildkite")
    sypd = None
    found_step_40 = False
    for line in job["raw_log_txt"].split("\n"):
        if (
            "n_steps_completed = 40" in line
            or "n_steps_completed = 41" in line
        ):
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

df = pl.DataFrame({"date": dates, "sypd": sypds, "run_from": run_locs})
df = df.sort("date")

# Add normalized SYPD column; normalize by the first non-null SYPD value
first_sypd = df["sypd"].drop_nulls()[0]
df = df.with_columns((pl.col("sypd") / first_sypd).alias("sypd_norm"))

print(df)

os.makedirs("results", exist_ok=True)
df.write_csv("results/amip.csv")

fig: Figure = df.drop_nulls().to_pandas().plot(
    x="date",
    y="sypd_norm",
    title="Flagship AMIP benchmark (16 horiz. elem.)",
    backend="plotly",
)  # type: ignore

# Use lines and markers
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis_title="Normalized SYPD", xaxis_title="Date")

# Save as PNG and JSON
os.makedirs("figures", exist_ok=True)
fig.write_image("figures/amip-sypd.png")
fig.write_json("figures/amip-sypd.json")
fig